<br>

## 2월7일 삼성전자 주가 예측

In [1]:
# 데이터 준비 
import pandas as pd
import numpy as np

samsung_data = pd.read_csv('samsung.csv', encoding='cp949', thousands=',')
samsung_data = pd.DataFrame(samsung_data).sort_index(ascending=False)

samsung_data.head()

,일자,시가,고가,저가,종가,거래량
425,2018-05-04,53000,53900,51800,51900,39565391
424,2018-05-08,52600,53200,51900,52600,23104720
423,2018-05-09,52600,52800,50900,50900,16128305
422,2018-05-10,51700,51700,50600,51600,13905263
421,2018-05-11,52000,52200,51200,51300,10314997


In [2]:
del(samsung_data['일자'])
samsung = np.array(samsung_data)

In [3]:
samsung_target = pd.read_excel('samsung0206.xlsx', encoding='cp949', thousands=',')
samsung_target = pd.DataFrame(samsung_target).sort_index(ascending=False)

del(samsung_target['Unnamed: 0'])
predict = np.array(samsung_target)
predict = predict.reshape(1,4,5)

In [4]:
# 데이터 셋 준비
x = []
y = []

# print(samsung_data.iloc[1:6, :], samsung_data.iloc[6,3])  >>> 예시
for i in range(1, len(samsung)-4, +1):
    x_tmp = samsung[i:i+4, :]   # (5행 5열 데이터 뽑아서 넣어줘)
    y_tmp = samsung[i+4, 3]     # x데이터 바로 다음 행의 종가 데이터 알려줘
    
    x.append(x_tmp)
    y.append(y_tmp)
########################################################################
x = np.array(x)
y = np.array(y)

In [5]:
print(x.shape, y.shape, predict.shape)

(421, 4, 5) (421,) (1, 4, 5)


In [6]:
# Data split하기
from sklearn.model_selection import train_test_split

x_train, x_test , y_train, y_test = train_test_split(x, y,
                                                     test_size=0.4,
                                                     shuffle = False)
x_test, x_val , y_test, y_val = train_test_split(x_test, y_test,
                                                     test_size=0.5,
                                                     shuffle = False)

In [7]:
# 2. 모델 구성
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

model = Sequential()

model.add(LSTM(32, input_shape=(4,5), activation='tanh'))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))
model.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                4864      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                1088      
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 520       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 9         
Total params: 7,009
Trainable params: 7,009
Non-trainable params: 0
____________________________________________________

In [8]:
#3. 훈련
model.compile(loss='mse', optimizer='adam', metrics=['mae'])
model.fit(x_train, y_train, epochs=100, batch_size=10, verbose=1)


Epoch 1/100
252/252 [==============================] - 0s 1ms/step - loss: 2023998910.9841 - mae: 44896.0586
Epoch 2/100
252/252 [==============================] - 0s 261us/step - loss: 2023984289.5238 - mae: 44895.9062
Epoch 3/100
252/252 [==============================] - 0s 284us/step - loss: 2023963073.0159 - mae: 44895.6680
Epoch 4/100
252/252 [==============================] - 0s 281us/step - loss: 2023906954.1587 - mae: 44895.0430
Epoch 5/100
252/252 [==============================] - 0s 283us/step - loss: 2023750138.9206 - mae: 44893.2969
Epoch 6/100
252/252 [==============================] - 0s 263us/step - loss: 2023329055.4921 - mae: 44888.6055
Epoch 7/100
252/252 [==============================] - 0s 273us/step - loss: 2022366267.9365 - mae: 44877.9062
Epoch 8/100
252/252 [==============================] - 0s 250us/step - loss: 2020362293.8413 - mae: 44855.5391
Epoch 9/100
252/252 [==============================] - 0s 265us/step - loss: 2016872154.4127 - mae: 44816.6055
Ep

252/252 [==============================] - 0s 213us/step - loss: 130160524.6349 - mae: 9209.8984
Epoch 73/100
252/252 [==============================] - 0s 217us/step - loss: 124096131.0159 - mae: 8953.8975
Epoch 74/100
252/252 [==============================] - 0s 213us/step - loss: 116015622.9365 - mae: 8803.7227
Epoch 75/100
252/252 [==============================] - 0s 194us/step - loss: 152266060.3175 - mae: 10111.3750
Epoch 76/100
252/252 [==============================] - 0s 130us/step - loss: 115119463.9365 - mae: 8978.5527
Epoch 77/100
252/252 [==============================] - 0s 264us/step - loss: 122969191.3333 - mae: 8998.7197
Epoch 78/100
252/252 [==============================] - 0s 130us/step - loss: 118735206.7460 - mae: 8751.6641
Epoch 79/100
252/252 [==============================] - 0s 184us/step - loss: 129975993.6190 - mae: 9410.0303
Epoch 80/100
252/252 [==============================] - 0s 183us/step - loss: 159905088.4127 - mae: 10116.9541
Epoch 81/100
252/252 

In [9]:
#4. 평가예측
evaluation = model.evaluate(x_test, y_test, batch_size=1)
print("evaluation:{}".format(evaluation))

84/84 [==============================] - 0s 1ms/step
evaluation:[11863795.228794644, 2986.041748046875]


In [32]:
# RMSE 만들기
from sklearn.metrics import mean_squared_error
y_predict = model.predict(predict, batch_size=1)
y_test_predict = model.predict(x_test, batch_size=1)

In [17]:
print("종가(test) : {}".format(y_test_predict[1]))
print("종가(pred) : {}".format(y_predict))

종가(test) : [42263.69]
종가(pred) : [[42263.69]]
